# Set covering implementation

In [192]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
import numpy as np

In [193]:
PROBLEM_SIZE = 20
NUM_SETS = 100
SET_PROBABILITY = 0.3
SETS = tuple(
    np.array([random() < SET_PROBABILITY for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple('State', ['taken', 'not_taken'])

## Utility functions
These are utility functions:
* goal_check: check if the current state covers all the positions
* distance: compute how many positions are covered by the current set
* count_taken_sets: count the actual number of positions covered by the set

In [194]:
def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def goal_check(state):
    return np.all(covered(state))

def distance(state):
    return PROBLEM_SIZE - sum(covered(state))

def count_taken_sets(state):
    return len(state.taken)

assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Probelm not solvable"


## Search function
This is a function to implement a search for set covering 
It allows to specify both the data structure on which to memorize the frontier and the priority function to use on that.

In [195]:
def set_covering_search(state=None, frontier=None, priority_func= None):
    
    if state is None:
        state = State(set(), set(range(NUM_SETS)))
    if frontier is None:
        frontier = PriorityQueue()
    if priority_func is None:
        priority_func = lambda _: None
        
    frontier.put((priority_func(state), state))
    counter = 0
    _, current_state = frontier.get()
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((priority_func(new_state), new_state))
        _, current_state = frontier.get()
    
    print(
        f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
    )
    print(f"Solution: {current_state}")

In [196]:
print("Breadth-first search")
set_covering_search(frontier=SimpleQueue())
print("Depth-first search")
set_covering_search(frontier=LifoQueue())
print("Greedy best-first search")
set_covering_search(frontier=PriorityQueue(), priority_func=distance)

Breadth-first search
Solved in 72,423 steps (3 tiles)
Solution: State(taken={96, 43, 6}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99})
Depth-first search
Solved in 7 steps (7 tiles)
Solution: State(taken={96, 97, 98, 99, 93, 94, 95}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92})
Greedy best-first search
Solved in 3 steps (3 t

## Lab 1
To address the lab1 request to make an A* search algorithm, the 2 functions to define the estimated cost for each node are:
* $ h(n) $: heuristic function that compute the cost to get from the actual node n to the goal state. it is required that it computes an optimistic previsions w.r.t the effective distance from the goal.
* $ g(n) $: actual cost function that compute the cost to reach the actual node n; the 'count_taken_sets' is used for that

The resulting priority function according to the queue is ordered is $ f(n) =  h(n) + g(n) $ 

### Considerations
* Initially, I tried to use the 'distance' function as a heuristic function, but the execution showed that it is pessimistic compared to the real distance to the goal:
    * Given n missing positions, this heuristic states that we will need at least n more sets to cover them, which is a way of saying that a set can cover at most one position
    *  Below the execution having the distance function as a heuristic that shows how using a pessimistic function does not allow for much improvement w.r.t. the Greedy best-first strategy and does not provide always an optimal solution, despite it is quite fast

In [197]:
print("A* search with pessimistic function")
set_covering_search(frontier=PriorityQueue(), priority_func=lambda state: count_taken_sets(state) + distance(state))

A* search with pessimistic function
Solved in 3 steps (3 tiles)
Solution: State(taken={8, 33, 60}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99})


* Below there are 3 optimistic heuristic functions proposed in class by the Professor

In [198]:
from math import ceil


def h(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken 

print("A* search with h")
set_covering_search(frontier=PriorityQueue(), priority_func=lambda state: count_taken_sets(state) + h(state))
print("A* search with h2")
set_covering_search(frontier=PriorityQueue(), priority_func=lambda state: count_taken_sets(state) + h2(state))
print("A* search with h3")
set_covering_search(frontier=PriorityQueue(), priority_func=lambda state: count_taken_sets(state) + h3(state))



A* search with h
Solved in 619 steps (3 tiles)
Solution: State(taken={72, 33, 39}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99})
A* search with h2
Solved in 40 steps (3 tiles)
Solution: State(taken={33, 83, 61}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99})
A* search with h3
Solved in 40 steps (3 tiles)
Solutio

* Then i tried to improve the heuristic function proposed by the professor by changing the number of sets considered to compute the candidates list in the h3 function, by remove all the sets that are already taken since their value is equal to 0 

In [199]:
def h3_improved(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(SETS[s], np.logical_not(already_covered))) for s in state.not_taken), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

* Below I make a comparison considering a bigger PROBLEM_SIZE

In [200]:
import time

PROBLEM_SIZE = 30
NUM_SETS = 100
SET_PROBABILITY = 0.3
SETS = tuple(
    np.array([random() <SET_PROBABILITY for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)

assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Probelm not solvable"

print("A* search with h3")
start_time = time.time()
set_covering_search(frontier=PriorityQueue(), priority_func=lambda state: count_taken_sets(state) + h3(state))
print(f"Execution time of A* search with h3 not improved: {time.time() - start_time}")
print("A* search with h3 improved")
start_time = time.time()
set_covering_search(frontier=PriorityQueue(), priority_func=lambda state: count_taken_sets(state) + h3_improved(state))
print(f"Execution time of A* search with h3 improved: {time.time() - start_time}")


A* search with h3
Solved in 664 steps (4 tiles)
Solution: State(taken={81, 66, 74, 5}, not_taken={0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99})
Execution time of A* search with h3 not improved: 19.107330083847046
A* search with h3 improved
Solved in 664 steps (4 tiles)
Solution: State(taken={81, 66, 74, 5}, not_taken={0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93,

* The improvement lead to a minor computation time w.r.t. to the old h3 despite the result is the same for both. Obviously to be the improvement remarkable is necessary to test over very larges PROBLEM_SIZES and NUM_SETS
